# Advent of Code 2022

https://adventofcode.com/2022

In [1]:
# A few standard library imports
import itertools, math, collections, os, functools, json
# nice printing
from IPython.display import display, clear_output, HTML

In [2]:
def getData(day):
    if os.path.exists("data-2022/day%dinput.txt" %day):
        with open("data-2022/day%dinput.txt" %day, 'r') as f:
            return f.read()
    import browsercookie
    import urllib.request
    cj = browsercookie.firefox()
    opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cj))
    assert b'watercrossing' in opener.open("https://adventofcode.com/2022/").read()
    data = opener.open("https://adventofcode.com/2022/day/%d/input" %day).read().decode('utf-8')
    with open("data-2022/day%dinput.txt" %day, 'w') as f:
        f.write(data)
    return data

## Day 1

In [3]:
day1data = [[int(y) for y in x.split()] for x in getData(1).split("\n\n")]

In [4]:
max(sum(x) for x in day1data)

64929

In [5]:
sum(sorted(sum(x) for x in day1data)[-3:])

193697

## Day 2

In [6]:
day2data = [(ord(x.split(" ")[0]) - 64, ord(x.split(" ")[1]) - 87) for x in getData(2).split("\n") if x]

In [7]:
sum(x[1] for x in day2data) + sum(any((x[1] - x[0]) == y for y in (1, -2))*6 + (x[1] == x[0])*3 for x in day2data)

12645

In [8]:
def day2Choice(oth, obj):
    if obj == 2: return oth
    if obj == 1: return oth - 1 if oth > 1 else 3
    if obj == 3: return oth + 1 if oth < 3 else 1

In [9]:
sum(day2Choice(*x) for x in day2data) + sum((x[1] - 1)*3 for x in day2data)

11756

## Day 3

In [10]:
day3data = [[ord(y) - 96 if ord(y) > 96 else ord(y) - 38 for y in x] for x in getData(3).split("\n") if x]

In [11]:
sum(sum(set(x[:len(x)//2]) & set(x[len(x)//2:])) for x in day3data)

8053

In [12]:
sum(sum(set(day3data[i]) & set(day3data[i+1]) & set(day3data[i+2])) for i in range(0, len(day3data), 3))

2425

## Day 4

In [13]:
day4data = [[range(int(y.split("-")[0]), int(y.split("-")[1]) + 1) for y in x.split(",")] for x in getData(4).split("\n") if x]

In [14]:
sum(len(set(x[0]).union(set(x[1]))) == max(len(x[0]), len(x[1])) for x in day4data)

651

In [15]:
sum(len(set(x[0]).intersection(set(x[1]))) > 0 for x in day4data)

956

## Day 5

In [16]:
day5stacks =  getData(5).split("\n\n")[0].split("\n")[:-1]
day5stacks = [[x[i] for x in day5stacks if x[i] != ' '] for i in range(1, len(day5stacks[0]), 4)]

In [17]:
day5instructions = [tuple(int(x.split(" ")[i]) for i in [1, 3, 5]) for x in getData(5).split("\n\n")[1].split("\n") if x]

In [18]:
for num, origin, dest in day5instructions:
    for i in range(num):
        day5stacks[dest - 1].insert(0, day5stacks[origin - 1].pop(0))
"".join(x[0] for x in day5stacks)

'SVFDLGLWV'

In [19]:
day5stacks =  getData(5).split("\n\n")[0].split("\n")[:-1]
day5stacks = [[x[i] for x in day5stacks if x[i] != ' '] for i in range(1, len(day5stacks[0]), 4)]

In [20]:
for num, origin, dest in day5instructions:
    for i in range(num):
        day5stacks[dest - 1].insert(i, day5stacks[origin - 1].pop(0))
"".join(x[0] for x in day5stacks)

'DCVTCVPCL'

## Day 6

In [21]:
day6data = getData(6).strip()

In [22]:
next(i for i in range(4, len(day6data)) if len(set(day6data[i-4:i])) == 4)

1282

In [23]:
next(i for i in range(14, len(day6data)) if len(set(day6data[i-14:i])) == 14)

3513

## Day 7

In [24]:
day7data = getData(7).strip().split("\n")

In [25]:
class Folder(object):
    def __init__(self, name, parent):
        self.name = name
        self.files = {} # name: size
        self.folders = {} # name: Folder objects
        self.parent = parent
    def prettyString(self):
        return"\n".join(self._format())
    def _format(self):
        s = ["- %s (dir)" %self.name]
        s.extend("  " + y for x in self.folders.values() for y in x._format())
        s.extend("  - %s (file, size=%d)" %(fn, size) for fn, size in self.files.items())
        return s
    def folderSize(self):
        return sum(self.files.values()) + sum(x.folderSize() for x in self.folders.values())
    def getChildFolders(self):
        return itertools.chain([self], *(x.getChildFolders() for x in self.folders.values()))
        
def parseStream(day7data):
    root = cwd = Folder('/', None)
    i = 1
    day7datalen = len(day7data)
    while i < day7datalen:
        line = day7data[i]
        if line == '$ ls':
            while i+1 < day7datalen and not (line := day7data[i+1]).startswith("$ "):
                if line.startswith("dir "):
                    cwd.folders[line.split("dir ", 1)[1]] = Folder(line.split("dir ", 1)[1], cwd)
                else:
                    cwd.files[line.split(" ", 1)[1]] = int(line.split(" ", 1)[0])
                i += 1
        elif line == '$ cd ..':
            cwd = cwd.parent
        elif line == '$ cd /':
            cwd = root
        elif line.startswith("$ cd "):
            cwd = cwd.folders[line.split("$ cd ", 1)[1]]
        else:
            print("ERROR: %s" %line)
        i += 1
    return root

In [26]:
root = parseStream(day7data)

In [27]:
print(root.prettyString())

- / (dir)
  - gts (dir)
    - grwwbrgz.wft (file, size=846)
    - mrnhn.psz (file, size=72000)
    - qvnbd.dqs (file, size=155241)
    - tndtmwfv (file, size=6655)
  - lwhbw (dir)
    - lrrl.lth (file, size=99946)
  - pcqjnl (dir)
    - gljcvm (dir)
      - tmwzlzn (file, size=264381)
    - lqwntmdg (dir)
      - jjfwr (dir)
        - cfhjvmh (dir)
          - gzfgc (dir)
            - cfhjvmh.wwh (file, size=134989)
      - rfqbmb (dir)
        - cbrvhz (dir)
          - wdtm.rjr (file, size=131072)
        - flcw (dir)
          - wlfwpb.wpg (file, size=216675)
        - mnd (dir)
          - hzzzzvmr.lsz (file, size=28976)
    - lrrl (dir)
      - cpmvnf (dir)
        - srtqvcv (dir)
          - mrnhn (dir)
            - fbrwd (dir)
              - nqth.gcn (file, size=163166)
      - dcfmtw (dir)
        - nzpdtfr (dir)
          - qwtwps (dir)
            - cmf (dir)
              - wdsjg.thm (file, size=73595)
          - vcthd (dir)
            - cfhjvmh (file, size=15016)
     

In [28]:
sum(s for f in root.getChildFolders() if (s := f.folderSize()) <= 100000)

1778099

In [29]:
desiredSize = 30000000 - (70000000 - root.folderSize())
sorted(s for f in root.getChildFolders() if (s := f.folderSize()) >= desiredSize)[0]

1623571

## Day 8

In [30]:
day8data = getData(8).strip().split("\n")

In [31]:
visibleTrees = list(itertools.product((0, len(day8data)), range(len(day8data[0])))) + \
                list(itertools.product(range(1, len(day8data) - 1), (0, len(day8data[0]))))
DIR = ((0, 1), (1, 0), (0, -1), (-1, 0))
for i, j in itertools.product(range(1, len(day8data) - 1), range(1, len(day8data[0]) - 1)):
    for diri, dirj in DIR:
        iloc, jloc = i + diri, j + dirj
        isVisible = True
        while isVisible and iloc not in (-1, len(day8data)) and jloc not in (-1, len(day8data[0])):
            isVisible = int(day8data[i][j]) > int(day8data[iloc][jloc])
            iloc, jloc = iloc + diri, jloc + dirj
        if isVisible:
            visibleTrees.append((i, j))
            break 
len(visibleTrees) 

1533

In [32]:
scenicScores = []
for i, j in itertools.product(range(1, len(day8data) - 1), range(1, len(day8data[0]) - 1)):
    thisScore = []
    for diri, dirj in ((0, 1), (1, 0), (0, -1), (-1, 0)):
        iloc, jloc = i + diri, j + dirj
        while iloc not in (-1, len(day8data)) and jloc not in (-1, len(day8data[0])) and \
                int(day8data[i][j]) > int(day8data[iloc][jloc]):
            iloc, jloc = iloc + diri, jloc + dirj
        thisScore.append(abs(min(max(iloc, 0), len(day8data) -1) - i) + abs(min(max(jloc, 0), len(day8data[0]) -1) - j))
    scenicScores.append(functools.reduce(lambda x, y: x*y, thisScore, 1))
max(scenicScores)

345744

## Day 9

In [33]:
day9data = [(x.split(" ")[0], int(x.split(" ")[1])) for x in getData(9).strip().split("\n")]

In [34]:
tailPositions = [[0, 0]]
headPosition = [0, 0]
for direction, steps in day9data:
    for i in range(steps):
        newTail = [x for x in tailPositions[-1]]
        if direction in ('U', 'D'):
            headPosition[1] += 1 if direction == 'U' else -1
            if abs(yOff := headPosition[1] - newTail[1]) > 1:
                newTail[1] += int(yOff/abs(yOff))
                if (xOff := headPosition[0] - newTail[0]) != 0:
                    newTail[0] += int(xOff/abs(xOff))
        elif direction in ('R', 'L'):
            headPosition[0] += 1 if direction == 'R' else -1
            if abs(xOff := headPosition[0] - newTail[0]) > 1:
                newTail[0] += int(xOff/abs(xOff))
                if (yOff := headPosition[1] - newTail[1]) != 0:
                    newTail[1] += int(yOff/abs(yOff))
        tailPositions.append(newTail)

In [35]:
len(set(tuple(x) for x in tailPositions))

6087

In [36]:
knotPositions = [[0, 0] for x in range(10)]
tailPositions = [[0, 0]]
for direction, steps in day9data:
    for i in range(steps):
        if direction in ('U', 'D'):
            knotPositions[0][1] += 1 if direction == 'U' else -1
        elif direction in ('R', 'L'):
            knotPositions[0][0] += 1 if direction == 'R' else -1           
        for i in range(0, 9):
            if abs(yOff := knotPositions[i][1] - knotPositions[i + 1][1]) > 1:
                knotPositions[i + 1][1] += int(yOff/abs(yOff))
                if (xOff := knotPositions[i][0] - knotPositions[i + 1][0]) != 0:
                    knotPositions[i + 1][0] += int(xOff/abs(xOff))
            if abs(xOff := knotPositions[i][0] - knotPositions[i + 1][0]) > 1:
                knotPositions[i + 1][0] += int(xOff/abs(xOff))
                if (yOff := knotPositions[i][1] - knotPositions[i + 1][1]) != 0:
                    knotPositions[i + 1][1] += int(yOff/abs(yOff))
        tailPositions.append([x for x in knotPositions[-1]])
        

In [37]:
len(set(tuple(x) for x in tailPositions))

2493

In [38]:
[x([z[i] for z in tailPositions]) for x, i in itertools.product([min, max], [0, 1])]

[-204, -7, 65, 207]

In [39]:
## hmm, a 269 x 214 grid is too large to visualise really :(

## Day 10

In [40]:
day10data = [(x.split(" ")[0], int(x.split(" ")[1]) if len(x.split(" ")) > 1 else None) 
             for x in getData(10).strip().split("\n")]

In [41]:
state = [1]
for op, num in day10data:
    if op == 'noop':
        state.append(state[-1])
    elif op == 'addx':
        state.extend([state[-1], state[-1] + num])

In [42]:
sum([i*state[i-1] for i in range(20,len(state), 40)])

14920

In [43]:
print("\n".join("".join(["#" if abs(s - (i % 40)) < 2 else " " 
                         for i, s in enumerate(state)])[(j*40):(j+1)*40] for j in range(len(state)//40)))

###  #  #  ##   ##   ##  ###  #  # #### 
#  # #  # #  # #  # #  # #  # #  #    # 
###  #  # #    #  # #    ###  #  #   #  
#  # #  # #    #### #    #  # #  #  #   
#  # #  # #  # #  # #  # #  # #  # #    
###   ##   ##  #  #  ##  ###   ##  #### 


## Day 11

In [44]:
def parseOp(op):
    if op == 'old * old': 
        return lambda x: x*x
    elif len(z := op.split('old * ')) > 1:
        return lambda x: x * int(z[1])
    elif len(z := op.split('old + ')) > 1:
        return lambda x: x + int(z[1])
    else:
        raise Exception("unknown op %s" %op)

In [45]:
day11Data = [{'op' : parseOp((y := x.split("\n"))[2].split(" = ")[1]),
              'initialItems': [int(z) for z in y[1].split(": ")[1].split(", ")],  
              'test' : int(y[3].split(" by ")[1]),
              'true': int(y[4].split('to monkey ')[1]),
              'false': int(y[5].split('to monkey ')[1])} 
             for x in getData(11).split("\n\n")]

In [46]:
for m in day11Data:
    m['items'] = [x for x in m['initialItems']]
    m['count'] = 0
for r in range(20):
    for m in day11Data:
        while m['items']:
            m['count'] += 1
            i = m['op'](m['items'].pop(0)) // 3
            day11Data[m['true'] if i % m['test'] == 0 else m['false']]['items'].append(i)

In [47]:
[a*b for a, b in [sorted([x['count'] for x in day11Data])[-2:]]][0]

54752

In [48]:
lcm = functools.reduce(lambda x, y: x*y, [x['test'] for x in day11Data], 1)

In [49]:
for m in day11Data:
    m['items'] = [x for x in m['initialItems']]
    m['count'] = 0
for r in range(10000):
    for m in day11Data:
        while m['items']:
            m['count'] += 1
            i = m['op'](m['items'].pop(0)) % lcm
            day11Data[m['true'] if i % m['test'] == 0 else m['false']]['items'].append(i)

In [50]:
[a*b for a, b in [sorted([x['count'] for x in day11Data])[-2:]]][0]

13606755504

## Day 12

In [51]:
day12Data = getData(12).strip().split("\n")

In [52]:
startX, startY = [(xi, yi) for xi in range(len(day12Data)) for yi in range(len(day12Data[0])) if day12Data[xi][yi] == 'S'][0]
endX, endY = [(xi, yi) for xi in range(len(day12Data)) for yi in range(len(day12Data[0])) if day12Data[xi][yi] == 'E'][0]

In [53]:
distanceMap = [[0 if day12Data[x][y] == 'S' else -1 for y in range(len(day12Data[0]))] for x in range(len(day12Data))]
toprocess = [(startX, startY)]
while toprocess:
    xi, yi = toprocess.pop(0)
    for dx, dy in ((0, 1), (1, 0), (0, -1), (-1, 0)):
        if (xi + dx in range(len(day12Data)) and yi + dy in range(len(day12Data[0])) 
            and ((day12Data[dx+xi][dy+yi] in ('a', 'b') and day12Data[xi][yi]  == 'S') or
                 (day12Data[dx+xi][dy+yi] == 'E' and day12Data[xi][yi]) in ('z', 'y') or 
                 (day12Data[dx+xi][dy+yi] != 'E' and ord(day12Data[dx+xi][dy+yi]) - ord(day12Data[xi][yi]) < 2))):
            if distanceMap[dx+xi][dy+yi] == -1 or distanceMap[dx+xi][dy+yi] > distanceMap[xi][yi] + 1:
                distanceMap[dx+xi][dy+yi] = distanceMap[xi][yi] + 1
                toprocess.append((dx+xi, dy+yi))

In [54]:
distanceMap[endX][endY]

447

In [55]:
distanceMap = [[0 if day12Data[x][y] in ('S', 'a') else -1 for y in range(len(day12Data[0]))] for x in range(len(day12Data))]
toprocess = [(x, y) for x in range(len(day12Data)) for y in range(len(day12Data[0])) if day12Data[x][y] in ('S', 'a')]
while toprocess:
    xi, yi = toprocess.pop(0)
    for dx, dy in ((0, 1), (1, 0), (0, -1), (-1, 0)):
        if (xi + dx in range(len(day12Data)) and yi + dy in range(len(day12Data[0])) 
            and ((day12Data[dx+xi][dy+yi] in ('a', 'b') and day12Data[xi][yi]  == 'S') or
                 (day12Data[dx+xi][dy+yi] == 'E' and day12Data[xi][yi]) in ('z', 'y') or 
                 (day12Data[dx+xi][dy+yi] != 'E' and ord(day12Data[dx+xi][dy+yi]) - ord(day12Data[xi][yi]) < 2))):
            if distanceMap[dx+xi][dy+yi] == -1 or distanceMap[dx+xi][dy+yi] > distanceMap[xi][yi] + 1:
                distanceMap[dx+xi][dy+yi] = distanceMap[xi][yi] + 1
                toprocess.append((dx+xi, dy+yi))

In [56]:
distanceMap[endX][endY]

446

## Day 13 

In [57]:
day13Data = [[json.loads(y) for y in x.split("\n")] for x in getData(13).strip().split("\n\n")]

In [58]:
def compare(x, y):
    """-1 left lower, 0 equal, 1 right lower"""
    if isinstance(x, int):
        if isinstance(y, int):
            if x < y: return -1
            if x == y: return 0
            return 1
        else:
            return compare([x], y)
    else:
        if isinstance(y, int):
            return compare(x, [y])
        else:
            if not x and not y: return 0
            if not x and y: return -1
            if not y and x: return 1
            for i in range(max(len(x), len(y))):
                ret = compare(x[i], y[i])
                if ret != 0: return ret
                if i+1 == len(x):
                    if i+1 == len(y): return 0
                    return -1
                if i+1 == len(y): return 1

In [59]:
sum(i+1 for i,x in enumerate(day13Data) if compare(x[0], x[1]) == -1)

5503

In [60]:
day13Sorted = sorted([x for y in day13Data for x in y] + [[[2]], [[6]]], key=functools.cmp_to_key(compare))

In [61]:
(day13Sorted.index([[2]]) + 1) * (day13Sorted.index([[6]]) + 1)

20952

## Day 14

In [62]:
day14Data = [[[int(z) for z in y.split(",")] for y in x.split(" -> ")] for x in getData(14).strip().split("\n")]

In [63]:
xmin, xmax = min(y[0] for x in day14Data for y in x), max(y[0] for x in day14Data for y in x)
ymin, ymax = min(y[1] for x in day14Data for y in x), max(y[1] for x in day14Data for y in x)
xmin, xmax, ymin, ymax

(496, 584, 16, 157)

In [64]:
cave = [[' ' for yi in range(xmax-xmin+1)] for xi in range(ymax+1)]
for wall in day14Data:
    for (wallSX, wallSY), (wallEX, wallEY) in zip(wall, wall[1:]):
        xdir = (wallEX-wallSX)//abs(wallEX-wallSX) if wallEX-wallSX else 1
        ydir = (wallEY-wallSY)//abs(wallEY-wallSY) if wallEY-wallSY else 1
        for xi, yi in itertools.product(range(wallSX-xmin, wallEX-xmin + xdir, xdir), range(wallSY, wallEY + ydir, ydir)):
            cave[yi][xi] = '#'
cave[0][500-xmin] = '+'

In [65]:
print("\n".join(["".join(y) for y in cave]))

    +                                                                                    
                                                                                         
                                                                                         
                                                                                         
                                                                                         
                                                                                         
                                                                                         
                                                                                         
                                                                                         
                                                                                         
                                                                                         
          

In [66]:
i = 0
try:
    while True:
        s = [0, 500 - xmin] # (y, x) yay
        while True:
            if cave[s[0] + 1][s[1]] == ' ':
                s[0] += 1
            elif cave[s[0] + 1][s[1] - 1] == ' ':
                s = [s[0] + 1, s[1] - 1]
            elif cave[s[0] + 1][s[1] + 1] == ' ':
                s = [s[0] + 1, s[1] + 1]
            else:
                cave[s[0]][s[1]] = 'o'
                i += 1
                break
except IndexError:
    pass

In [67]:
print(i)

1513


In [68]:
print("\n".join(["".join(y) for y in cave]))

    +                                                                                    
                                                                                         
                                                                                         
                                                                                         
                                                                                         
                                                                                         
                                                                                         
                                                                                         
                                                                                         
                                                                                         
                                                                                         
          

In [69]:
cave = [[' ' for xi in range(ymax*2 + 4 + 1)] for yi in range(ymax+2+1)]
for wall in day14Data:
    for (wallSX, wallSY), (wallEX, wallEY) in zip(wall, wall[1:]):
        xdir = (wallEX-wallSX)//abs(wallEX-wallSX) if wallEX-wallSX else 1
        ydir = (wallEY-wallSY)//abs(wallEY-wallSY) if wallEY-wallSY else 1
        for xi, yi in itertools.product(range(wallSX + ymax - 500 + 2 , wallEX + ymax + xdir - 500 + 2, xdir), 
                                        range(wallSY, wallEY + ydir, ydir)):
            cave[yi][xi] = '#'
for xi in range(ymax*2 + 4 + 1):
    cave[ymax + 2][xi] = '#'
cave[0][(ymax + 1) + 1] = '+'


In [70]:
xmin, xmax,  ymin, ymax

(496, 584, 16, 157)

In [71]:
i = 0
try:
    while True:
        s = [0, (ymax + 1) + 1] # (y, x) yay
        while True:
            if cave[s[0] + 1][s[1]] == ' ':
                s[0] += 1
            elif cave[s[0] + 1][s[1] - 1] == ' ':
                s = [s[0] + 1, s[1] - 1]
            elif cave[s[0] + 1][s[1] + 1] == ' ':
                s = [s[0] + 1, s[1] + 1]
            else:
                i += 1
                if cave[s[0]][s[1]] == '+':
                    raise IndexError
                cave[s[0]][s[1]] = 'o'
                break
except IndexError:
    pass

In [72]:
i

22646

In [73]:
display(HTML("<style>pre { white-space: pre !important; }</style>"))
print("\n".join(["".join(y) for y in cave]))

                                                                                                                                                               +                                                                                                                                                               
                                                                                                                                                              ooo                                                                                                                                                              
                                                                                                                                                             ooooo                                                                                                                                                             
                                        

## Day 15

In [74]:
day15Data = [((int(x.split(", y=")[0].split("x=")[1]), int(x.split(", y=")[1].split(": ")[0])),
(int(x.split(", y=")[1].split("x=")[1]), int(x.split(", y=")[2]))) for x in getData(15).strip().split("\n")]

In [75]:
def simplifyRange(rs):
    outs = []
    for s, e in sorted([(x, y) for x, y in rs if x!= y]):
        if outs and outs[-1][1] + 1 > s:
            outs[-1] = (outs[-1][0], max(e, outs[-1][1]))
        else:
            outs.append((s, e))
    return outs

In [76]:
sum(xmax - xmin for xmin, xmax in simplifyRange(((sX - max(abs(sX-bX) + abs(sY-bY) - abs(sY-2000000), 0)), 
                                                 sX + max(abs(sX-bX) + abs(sY-bY) - abs(sY-2000000), 0)) 
                                                for (sX, sY), (bX, bY) in day15Data))

4883971

In [77]:
def isValid(x, y):
    return all(abs(sX-x) + abs(sY-y) > abs(sX-bX) + abs(sY-bY) for (sX, sY), (bX, bY) in day15Data)

In [78]:
### 17 seconds... that will do
onEdge = next((x, y) for (sX, sY), (bX, bY) in day15Data
              for x in range(min(max(0,sX -(abs(sX-bX) + abs(sY-bY) + 1)), 4000000),
                              min(max(0,sX + abs(sX-bX) + abs(sY-bY) + 1), 4000000))
          for y in [sY + (abs(sX-bX) + abs(sY-bY) + 1 - x + sX), sY - (abs(sX-bX) + abs(sY-bY) + 1 - x + sX)] 
          if y > -1 and y <= 4000000 
          and isValid(x, y))

In [79]:
onEdge, onEdge[1] + 4000000*onEdge[0]

((3172756, 2767556), 12691026767556)

## Day 16

In [80]:
day16Data = dict((x[6:8], (int(x.split("rate=")[1].split(";")[0]), x.split(" ",9)[9].split(", "))) 
                 for x in getData(16).strip().split("\n"))

In [81]:
# build a fully connected graph of distance costs
day16Graph = collections.defaultdict(dict)
for valve, (flow, dests) in day16Data.items():
    for dest in dests:
        day16Graph[valve][dest] = 1
        day16Graph[dest][valve] = 1
added = True
while added:
    added = False
    for start, dests in day16Graph.items():
        for dest, cost in list(dests.items()):
            for otherDest in day16Graph[dest]:
                if otherDest == start:
                    continue
                if otherDest in dests:
                    if dests[otherDest] > cost + day16Graph[dest][otherDest]:
                        added = True
                        dests[otherDest] = cost + day16Graph[dest][otherDest]
                else:
                    added = True
                    dests[otherDest] = cost + day16Graph[dest][otherDest]
# prune useless valves
for valve, (flow, dests) in day16Data.items():
    if flow == 0 and valve != 'AA':
        if valve in day16Graph: del day16Graph[valve]
        for nodes in day16Graph.values(): 
            if valve in nodes: del nodes[valve]

In [82]:
def releasePressure(graph, timeLeft, valvesOpen, currentLocation):
    if timeLeft == 0: return (0, [(timeLeft, currentLocation)])
    options = [releasePressure(graph, timeLeft - cost - 1, valvesOpen + [currentLocation], dest) 
               for dest, cost in graph[currentLocation].items() if dest not in valvesOpen and cost < timeLeft]
    bestOption = max(options, key=lambda x: x[0]) if options else (0, [(timeLeft, "remain here")])
    return (day16Data[currentLocation][0]*timeLeft + bestOption[0], [(timeLeft, currentLocation)] + bestOption[1]) 

In [83]:
releasePressure(day16Graph, 30, [], 'AA')

(1923,
 [(30, 'AA'),
  (27, 'NV'),
  (24, 'PS'),
  (21, 'FX'),
  (18, 'JM'),
  (15, 'KZ'),
  (12, 'UX'),
  (9, 'DO'),
  (6, 'PH'),
  (3, 'DG'),
  (0, 'RG')])

In [84]:
score, steps = releasePressure(day16Graph, 26, [], 'AA')
score2, steps2 = releasePressure(day16Graph, 26, [x[1] for x in steps], 'AA')
score + score2, list(itertools.zip_longest(steps, steps2))

(2594,
 [((26, 'AA'), (26, 'AA')),
  ((23, 'NV'), (23, 'NU')),
  ((20, 'PS'), (20, 'YA')),
  ((17, 'FX'), (17, 'EJ')),
  ((14, 'JM'), (13, 'XK')),
  ((11, 'KZ'), (3, 'RG')),
  ((8, 'UX'), (0, 'DG')),
  ((5, 'DO'), None),
  ((2, 'PH'), None),
  ((2, 'remain here'), None)])